In [1]:
from aux.aux_functions import *

In [2]:
import openai
from openai import OpenAI

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

from IPython.display import display

In [4]:
import os

openai.api_key = os.environ["OPENAI_API_KEY"]

In [139]:
# Specify the path to your LaTeX file
#2310.20559
#latex_file_path = 'Files/2310.20559/main.tex'

name_file = "0701050/review.tex"

latex_file_path = 'Files/' + name_file

# Extract the LaTeX content
latex_content = extract_latex_content(latex_file_path, "\\startdocument", "\\enddocument")

# For demonstration, let's print the first characters to check if it worked
len(latex_content)

76800

In [140]:
replacements = {
        r'\\d': 'd',
        r'\\G': 'G',
        r'\\O': 'O',
        r'\\C': 'C',
        r'\\V': 'V',
        r'\\L': 'L',
        r'\\R': 'R',
        r'\\S': 'S',
        r'\\Mp': 'Mp',
        r'\\cN': 'N',
        r'\\cC': 'C',
        r'\\cP': 'P',
        r'\\cR': 'R',
        r'\\vo': 'V',
        r'\\Kahler': 'Kahler',
        r'\\flux': 'flux',
        r'\\KK': 'Kaluza-Klein',
        r'\\n': ' '
    }

brace_commands = [r'\\emph', r'\\footnote', r'\\it', r'\\em', r'\\ref', r'\\eqref']

simplified_text = replace_latex_commands(latex_content, replacements, brace_commands)
len(simplified_text)

76552

In [141]:
polished_text = clean_latex_content(simplified_text)
len(polished_text)

75561

In [143]:
no_char_in_chunk = 2000

chunks = chunk_text(polished_text, no_char_in_chunk)
chunks = [chunk.replace("%", "") for chunk in chunks]
print("The number of chunks is: " + str(len(chunks)) + ".")
chunks[23:25]

The number of chunks is: 45.


["The number of vacua $\\phi_*$ in an interval $I$ is given by \\begin{equation}\n\\label{ansatz}\nds^2 = g_{\\mu u} dx^{\\mu}dx^{ u} + R^2 \\tilde g_{mn}dy^m dy^n\n\\end{equation}6 where $\\theta(x):=1$ if $x>0$, $\\theta(x):=0$ if $x<0$. The integrand $delta(V') |V''|$ gives a contribution $+1$ for each critical point in $I$, while $\\theta(V'')$ restricts to actual minima. Now in the large $L$ limit, we can approximate the sum over $(N,M)$ by an integral, and write \\begin{equation}\n\\label{ansatz}\nds^2 = g_{\\mu u} dx^{\\mu}dx^{ u} + R^2 \\tilde g_{mn}dy^m dy^n\n\\end{equation}7 where $\\rho(\\phi)$ can be interpreted as a vacuum number density on moduli space. To evaluate the integral over $(N,M)$ at a given fixed $\\phi$, it is convenient to make the following linear change of variables $(N,M) \\to (v',v'')$: \\begin{equation}\n\\label{ansatz}\nds^2 = g_{\\mu u} dx^{\\mu}dx^{ u} + R^2 \\tilde g_{mn}dy^m dy^n\n\\end{equation}8 This change of variables has Jacobian $=1$, and the 

In [144]:
# Define the number of Q&A pairs you want to generate for each chunk
num_questions = 3

In [145]:
system_prompt_template = "You are an expert in theoretical physics. I will provide a text, and I'd like you to generate {num_questions} questions a researcher-level colleague might ask about the topic of the text. Then, provide detailed answers to these questions as an expeert in the field. Format your responses as follows: 'Question 1: [question here] Answer 1: [answer here]' and so on for all {num_questions} questions."

user_prompt_template = "Based on the following text, please generate the questions and answers as instructed. Text: {chunk}"

display(system_prompt_template)
display(user_prompt_template)


"You are an expert in theoretical physics. I will provide a text, and I'd like you to generate {num_questions} questions a researcher-level colleague might ask about the topic of the text. Then, provide detailed answers to these questions as an expeert in the field. Format your responses as follows: 'Question 1: [question here] Answer 1: [answer here]' and so on for all {num_questions} questions."

'Based on the following text, please generate the questions and answers as instructed. Text: {chunk}'

In [146]:
client = OpenAI()

# Initialize a list to store chunk and response pairs
chunk_responses = []

# Assume chunks_test is a list of text chunks you want to generate Q&A for
for chunk in chunks:
    system_prompt, user_prompt = generate_prompt(chunk, num_questions, system_prompt_template, user_prompt_template)
    
    # Call the OpenAI API once per chunk
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[system_prompt, user_prompt]
    )
    
    # Append both the chunk and its response to the chunk_responses list
    chunk_responses.append({
        "chunk": chunk,
        "response": response.choices[-1].message.content if response.choices else "No response"
    })


In [147]:
# Initialize lists to hold the DataFrame data
chunk_list = []
question_list = []
answer_list = []
sub_index = []

# Process each chunk-response pair
for item in chunk_responses:
    chunk = item['chunk']
    response = item['response']
    
    # Split the response into Q&As
    qas = response.split('\n\n')  # Assuming two newlines separate each Q&A pair
    for i, qa in enumerate(qas):
        # Attempt to split the QA into question and answer
        parts = qa.split('\nAnswer ')
        if len(parts) == 2:
            question = parts[0].replace('Question {}: '.format(i+1), '').strip()
            answer = parts[1].strip()
            chunk_list.append(chunk)
            question_list.append(question)
            answer_list.append(answer)
            sub_index.append(i+1)

In [148]:
# Create a MultiIndex from the chunks and their corresponding sub_index
multi_index = pd.MultiIndex.from_arrays([chunk_list, sub_index], names=['Chunk', 'Q&A Index'])
answer_list = [re.sub(r'^\d+: ', '', answer) for answer in answer_list]

# Construct the DataFrame
df_multi = pd.DataFrame({'Question': question_list, 'Answer': answer_list}, index=multi_index)

# Display the DataFrame
df_multi

Question  \
Chunk                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       Q&A Index                                                                                                                                                                                                                                             
String theory was first proposed as a candidate theory of quantum gravity in 1974 \cite{Scherk:1974mc}. Over the subsequent years, supersymmetric versions of the theory were developed, and arguments made that they were perturbatively finite \cite{Schwarz:1982jn}. The discoveries of anomaly cancellation \cite{Green:1984sg} and of quasi-realistic compactifications of the heterotic string \cite{Candelas:1985en} in 1984--85, a period sometimes referred to as the ``first superstring revolution,'' led very rapidly to a broad consensus among particle physicists that superstring theory was a viable contender for a ``theory of everything,'' describing all of fundamental physics. Now at this point, there were several good reasons to be wary of this claim. One was that there appeared to be many competing theories; besides the different varieties of superstrings, there was eleven-dimensional supergravity. Another was the fact that string theory was defined only as a perturbative expansion, which could only be used directly at weak coupling. Besides the evident fact that the real world includes strongly coupled QCD, various arguments had been made that constructing a completely realistic model would require non-perturbative physics, most notably at the stage of supersymmetry breaking \cite{Dine:1985he}. These doubts were addressed in a rather striking way in the ``second superstring revolution'' of 1994--97, in which it was convincingly argued that all of these theories are limits or aspects of one unified framework, usually now called ``string/M theory.'' The central idea, called duality, is that the strong coupling limit of one of the various string theories, can be equivalent (or dual) to another weakly coupled theory, one of the strings or else M theory, the eleven-dimens

In [149]:
qa_df = pd.DataFrame({
    'Question': question_list,
    'Answer': answer_list
})

qa_df = qa_df
qa_df

,Question,Answer
0,How did the second superstring revolution of 1994-97 address the doubts surrounding string theory being defined only as a perturbative expansion and the need for non-perturbative physics in constructing a completely realistic model?,"The second superstring revolution of 1994-97 addressed these doubts by convincingly arguing that all the different string theories and eleven-dimensional supergravity are actually limits or aspects of a unified framework called string/M theory. The central idea of duality emerged, demonstrating that the strong coupling limit of one string theory can be equivalent to another weakly coupled theory, leading to a more unified understanding of the subject."
1,"What were some of the key discoveries that led particle physicists to consider superstring theory as a viable contender for a ""theory of everything""?","Some key discoveries that propelled superstring theory's candidacy for a ""theory of everything"" included the development of supersymmetric versions of the theory, arguments for perturbative finiteness, discoveries of anomaly cancellations, and quasi-realistic compactifications of the heterotic string. These advancements collectively contributed to a broad consensus among physicists that superstring theory could potentially describe all of fundamental physics."
2,How did the concept of duality play a significant role in the development and acceptance of string/M theory as a unified framework encompassing various string theories and eleven-dimensional supergravity?,"The concept of duality was crucial in the evolution of string theory towards the unified framework of string/M theory. Duality showed that seemingly distinct string theories were actually different descriptions of the same underlying physics. For example, it revealed that the strongly coupled regime of one theory could be equivalent to the weakly coupled regime of another theory. This unification through duality helped reconcile the different versions of string theory and eleven-dimensional supergravity into a singular comprehensive framework, string/M theory, contributing to a deeper understanding of the subject."
3,What is the central idea of duality in the context of string/M theory as mentioned in the text?,"The central idea of duality in string/M theory refers to the concept that the strong coupling limit of one string theory can be equivalent to another theory that is weakly coupled, including M theory. This concept reveals that seemingly distinct theories are actually different limits or aspects of one unified framework."
4,"How did the ""second superstring revolution"" from 1994-97 contribute to theoretical physics as mentioned in the text?","The ""second superstring revolution"" during 1994-97 played a crucial role in theoretical physics by providing nonperturbative definitions for string/M theory and offering exact solutions for the effective Lagrangians of various supersymmetric field theories. It introduced the idea of duality, which established connections between different string theories and led to a unified framework known as string/M theory."
...,...,...
94,How does the text relate the search for inflationary measures in theoretical physics to analogous problems in particle physics?,"The text draws an analogy between the search for inflationary measures in theoretical physics and the hypothetical scenario of developing particle physics based on data at energies far below the natural scales of the Standard Model, suggesting that advances using precision information and very powerful supercomputers could potentially lead to inferences about the validity of certain theories even at energy scales well beyond what is experimentally accessible."
95,"In the context of the text, what role do asymptotically free gauge theories coupled to gravity play in advancing theoretical physics and potentially leading to the Standard Model?","The text suggests that asymptotically free gauge theories coupled to gravity 

In [152]:
qa_df.to_csv("Datasets/0701050.csv")
qa_df.to_parquet("Datasets/0701050.parquet")

In [153]:
df_test = pd.read_csv("Datasets/0701050.csv", index_col=0)
df_test

,Question,Answer
0,How did the second superstring revolution of 1994-97 address the doubts surrounding string theory being defined only as a perturbative expansion and the need for non-perturbative physics in constructing a completely realistic model?,"The second superstring revolution of 1994-97 addressed these doubts by convincingly arguing that all the different string theories and eleven-dimensional supergravity are actually limits or aspects of a unified framework called string/M theory. The central idea of duality emerged, demonstrating that the strong coupling limit of one string theory can be equivalent to another weakly coupled theory, leading to a more unified understanding of the subject."
1,"What were some of the key discoveries that led particle physicists to consider superstring theory as a viable contender for a ""theory of everything""?","Some key discoveries that propelled superstring theory's candidacy for a ""theory of everything"" included the development of supersymmetric versions of the theory, arguments for perturbative finiteness, discoveries of anomaly cancellations, and quasi-realistic compactifications of the heterotic string. These advancements collectively contributed to a broad consensus among physicists that superstring theory could potentially describe all of fundamental physics."
2,How did the concept of duality play a significant role in the development and acceptance of string/M theory as a unified framework encompassing various string theories and eleven-dimensional supergravity?,"The concept of duality was crucial in the evolution of string theory towards the unified framework of string/M theory. Duality showed that seemingly distinct string theories were actually different descriptions of the same underlying physics. For example, it revealed that the strongly coupled regime of one theory could be equivalent to the weakly coupled regime of another theory. This unification through duality helped reconcile the different versions of string theory and eleven-dimensional supergravity into a singular comprehensive framework, string/M theory, contributing to a deeper understanding of the subject."
3,What is the central idea of duality in the context of string/M theory as mentioned in the text?,"The central idea of duality in string/M theory refers to the concept that the strong coupling limit of one string theory can be equivalent to another theory that is weakly coupled, including M theory. This concept reveals that seemingly distinct theories are actually different limits or aspects of one unified framework."
4,"How did the ""second superstring revolution"" from 1994-97 contribute to theoretical physics as mentioned in the text?","The ""second superstring revolution"" during 1994-97 played a crucial role in theoretical physics by providing nonperturbative definitions for string/M theory and offering exact solutions for the effective Lagrangians of various supersymmetric field theories. It introduced the idea of duality, which established connections between different string theories and led to a unified framework known as string/M theory."
...,...,...
94,How does the text relate the search for inflationary measures in theoretical physics to analogous problems in particle physics?,"The text draws an analogy between the search for inflationary measures in theoretical physics and the hypothetical scenario of developing particle physics based on data at energies far below the natural scales of the Standard Model, suggesting that advances using precision information and very powerful supercomputers could potentially lead to inferences about the validity of certain theories even at energy scales well beyond what is experimentally accessible."
95,"In the context of the text, what role do asymptotically free gauge theories coupled to gravity play in advancing theoretical physics and potentially leading to the Standard Model?","The text suggests that asymptotically free gauge theories coupled to gravity 

In [157]:
from sklearn.utils import shuffle

directory_path = 'Datasets/'

# Initialize a list to hold the dataframes
dataframes = []

# Iterate through all files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        # Load each CSV file into a DataFrame, excluding the index
        df_temp = pd.read_csv(os.path.join(directory_path, filename), index_col=None)
        # Append the DataFrame to the list
        dataframes.append(df_temp)

# Concatenate all the DataFrames into a single DataFrame
concatenated_df = pd.concat(dataframes, ignore_index=True)

# Shuffle the rows of the concatenated DataFrame
shuffled_df = shuffle(concatenated_df, random_state=42)

# Reset the index to get rid of the old indexing
shuffled_df.reset_index(drop=True, inplace=True)

shuffled_df

,Unnamed: 0,Question,Answer
0,247,How do the bosonic generators in the component description of supersymmetry relate to the supersymmetry algebra?,"In the component description of supersymmetry, the bosonic generators are defined as $\delta_\xi = \xi Q + \bar{\xi} \bar{Q}$. The algebra defined with commutators of these generators is equivalent to the supersymmetry algebra. By calculating the commutators and ensuring that the algebra is represented on the component fields, one can demonstrate the consistency of the supersymmetry transformations at the component level."
1,196,"How is the transition between different vacuum states, such as dS to AdS or dS to dS, estimated in the context of string theory compactifications as mentioned in the text?","The transition rates between different vacuum states, such as from de Sitter (dS) to Anti-de Sitter (AdS) or dS to dS, are estimated based on the potential for the volume modulus that connects these vacua. The probability amplitude for the transition, denoted as $g_s^7$ where $g_s^8$ is the value of the scalar potential at the dS minimum, reflects the likelihood of the transition occurring. The corresponding lifetime $g_s^9$ of such transitions is typically exponentially small compared to the Poincaré recurrence time, which is a reassuring feature."
2,38,Can you explain how the spectra generated for density perturbations and gravitational waves during inflation provide a distinctive signature in the CMB anisotropies?,"The spectra of density perturbations and gravitational waves produced during inflation have distinct patterns based on the physics of inflation. These patterns leave characteristic imprints on the temperature fluctuations and polarization of the CMB. By analyzing the anisotropies in the CMB, we can study these spectra, which in turn provide valuable insights into the early universe, the nature of inflation, and the fundamental forces and particles at play during that time."
3,781,How does the topology and geometry of Anti-de-Sitter space play a role in the AdS/CFT correspondence?,"Anti-de-Sitter space has the topology of a solid cylinder, where the curvature of space results in objects tending to fall towards the center of the cylinder, representing the lowest gravitational potential. The correspondence relates the bulk gravitational theory in this negatively curved space to a conformal field theory (CFT) living on the boundary. The real axis representing time and the radial direction in the bulk space correspond to energy scales in the CFT."
4,241,Can you explain the steps involved in rewriting the second matrix on the left-hand side (l.h.s) of the given equation using Pauli matrices and unit matrix relations?,"To rewrite the second matrix on the l.h.s of the equation, one can utilize the relations between Pauli matrices and the unit matrix to manipulate the expressions. By applying the Pauli matrices and their relationships with the unit matrix, it becomes possible to rewrite the matrix on the l.h.s of the equation in terms of the desired form."
...,...,...,...
2596,292,What is the significance of tracelessness of the energy-momentum tensor in the context of string theory and its equations of motion?,"In the context of string theory, the equation of motion is given by $T^{ab}=0$. An important feature is the tracelessness of the energy-momentum tensor, i.e., $T^a{}_a=0$ for any $h_{ab}$. This tracelessness holds as an identity independently of whether the field configuration obeys the equations of motion, and it generally follows from symmetry."
2597,101,"How can the concept of de Rham cohomology help distinguish between exact and closed forms in theoretical physics, particularly in the context of different gravitational constants?","The concept of de Rham cohomology provides a mathematical framework to differentiate between exact and closed forms in theoretical physics, specifically with respect to gravitational constants like $G_N$. In the given context, exact forms co